<a href="https://colab.research.google.com/github/Thomas-George-T/Retro-Reel-Recommender/blob/main/Retro_Reel_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Retrol Reel Recommender

## Data Acquisition

In [120]:
import pandas as pd

In [121]:
!wget -q -O 'retro.json' https://prosona.b-cdn.net/Movies/catalog-v2.txt
print("Data Downloaded!")

Data Downloaded!


In [122]:
retro_raw = pd.read_json('retro.json')
retro_raw.head()

,Movies
0,"{'ID': 'b70eec12b79148dfbc97ac3b69ca49bf', 'Ti..."
1,"{'ID': '1950e4a11304438dbfd4792a908fc76f', 'Ti..."
2,"{'ID': '37d8cae8d87f4dbc9cf8967454537afb', 'Ti..."
3,"{'ID': 'b99dd03c00cc4fb9b2d4ad26082de498', 'Ti..."
4,"{'ID': '5566ce75b5954f3dad323c2f154856a3', 'Ti..."


## Data Preprocessing

In [123]:
retro_raw_flatten = pd.json_normalize(retro_raw['Movies'])
retro_raw_flatten.head()

,ID,Title,ReleaseYear,Genres,Synopsis,Directors,Cast,Trivia,Keywords,TrailerLink,MovieLink,HorizontalPosterLink,VerticalPosterLink,SnapshotLink,SubtitlesLink,Offset
0,b70eec12b79148dfbc97ac3b69ca49bf,Angel On My Shoulder,1946,"[Adventure, Comedy, Fantasy]",The Devil arranges for a deceased gangster to ...,[Archie Mayo],"[Paul Muni, Anne Baxter, Claude Rains]",[Director Archie Mayo clashed repeatedly with ...,Devil Soul angel on my shoulder 1946 adventure...,https://prosona.b-cdn.net/Movies/Angel%20On%20...,https://prosona.b-cdn.net/Movies/Angel%20On%20...,https://prosona.b-cdn.net/Movies/Angel%20On%20...,https://prosona.b-cdn.net/Movies/Angel%20On%20...,https://prosona.b-cdn.net/Movies/Angel%20On%20...,,0.0
1,1950e4a11304438dbfd4792a908fc76f,Flirting with Fate,1916,"[Adventure, Comedy]","In the midst of an emotional depression, a man...",[Christy Cabanne],"[Douglas Fairbanks, Jewel Carmen, Howard Gaye]",[],hitman assasin killer escape flirting with fat...,https://prosona.b-cdn.net/Movies/Flirting%20Wi...,https://prosona.b-cdn.net/Movies/Flirting%20Wi...,https://prosona.b-cdn.net/Movies/Flirting%20Wi...,https://prosona.b-cdn.net/Movies/Flirting%20Wi...,https://prosona.b-cdn.net/Movies/Flirting%20Wi...,,0.0
2,37d8cae8d87f4dbc9cf8967454537afb,Detour,1945,"[Crime, Drama, Noir]",Chance events trap hitch-hiker Al Roberts in a...,[Edgar G. Ulmer],"[Tom Neal, Ann Savage, Claudia Drake]",[The budget PRC gave director Edgar G. Ulmer f...,detour 1945 crime drama noir edgar g. ulmer to...,https://prosona.b-cdn.net/Movies/Detour%20-%20...,https://prosona.b-cdn.net/Movies/Detour%20-%20...,https://prosona.b-cdn.net/Movies/Detour%20-%20...,https://prosona.b-cdn.net/Movies/Detour%20-%20...,https://prosona.b-cdn.net/Movies/Detour%20-%20...,https://prosona.b-cdn.net/Movies/Detour%20-%20...,0.0
3,b99dd03c00cc4fb9b2d4ad26082de498,The Big Trees,1952,"[Action, Romance, Western]",A Quaker colony tries to save the giant sequoi...,[Felix E. Feist],"[Kirk Douglas, Eve Miller, Patrice Wymore]",[According to Kirk Douglas in his autobiograph...,activism sequoia the big trees 1952 action rom...,https://prosona.b-cdn.net/Movies/The%20Big%20T...,https://prosona.b-cdn.net/Movies/The%20Big%20T...,https://prosona.b-cdn.net/Movies/The%20Big%20T...,https://prosona.b-cdn.net/Movies/The%20Big%20T...,https://prosona.b-cdn.net/Movies/The%20Big%20T...,,0.0
4,5566ce75b5954f3dad323c2f154856a3,The Jungle Book,1942,"[Action, Adventure]",\r\nA boy raised by wild animals tries to adap...,[Zoltan Korda],"[Sabu, Joseph Calleia, John Qualen]",[Rudyard Kipling's two Jungle Book anthologies...,rudyard kipling wolves family the jungle book ...,https://prosona.b-cdn.net/Movies/Jungle%20Book...,https://prosona.b-cdn.net/Movies/Jungle%20Book...,https://prosona.b-cdn.net/Movies/Jungle%20Book...,https://prosona.b-cdn.net/Movies/Jungle%20Book...,https://prosona.b-cdn.net/Movies/Jungle%20Book...,,0.0


## Feature Selection

In [124]:
rr = retro_raw_flatten[['ID','Title','Keywords']]
rr.head()

,ID,Title,Keywords
0,b70eec12b79148dfbc97ac3b69ca49bf,Angel On My Shoulder,Devil Soul angel on my shoulder 1946 adventure...
1,1950e4a11304438dbfd4792a908fc76f,Flirting with Fate,hitman assasin killer escape flirting with fat...
2,37d8cae8d87f4dbc9cf8967454537afb,Detour,detour 1945 crime drama noir edgar g. ulmer to...
3,b99dd03c00cc4fb9b2d4ad26082de498,The Big Trees,activism sequoia the big trees 1952 action rom...
4,5566ce75b5954f3dad323c2f154856a3,The Jungle Book,rudyard kipling wolves family the jungle book ...


Formatting the Keywords since that is what we will be using to derive our model

In [125]:
rr['Keywords'] = rr['Keywords'].str.lower()
rr.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,ID,Title,Keywords
0,b70eec12b79148dfbc97ac3b69ca49bf,Angel On My Shoulder,devil soul angel on my shoulder 1946 adventure...
1,1950e4a11304438dbfd4792a908fc76f,Flirting with Fate,hitman assasin killer escape flirting with fat...
2,37d8cae8d87f4dbc9cf8967454537afb,Detour,detour 1945 crime drama noir edgar g. ulmer to...
3,b99dd03c00cc4fb9b2d4ad26082de498,The Big Trees,activism sequoia the big trees 1952 action rom...
4,5566ce75b5954f3dad323c2f154856a3,The Jungle Book,rudyard kipling wolves family the jungle book ...


In [126]:
from sklearn.feature_extraction.text import CountVectorizer

In [127]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(rr['Keywords'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(rr.Title)
indices

0                        Angel On My Shoulder
1                          Flirting with Fate
2                                      Detour
3                               The Big Trees
4                             The Jungle Book
5                              The Lost World
6                                    Suddenly
7                        Bloody Pit Of Horror
8                  Charlie Chaplin's The Rink
9                                Texas Terror
10                              The Iron Mask
11                               Barefoot Boy
12                                 Borderline
13             Charlie Chaplin's The Vagabond
14                            Galloping Romeo
15                            His Girl Friday
16                            The Star Packer
17                             Scarlet Street
18    Charlie Chaplin's A Burlesque On Carmen
19             Charlie Chaplin's The Pawnshop
20                                    Manfish
21                          Carniv

In [128]:
from sklearn.metrics.pairwise import cosine_similarity

In [129]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.11111111, 0.        , ..., 0.06085806, 0.05407381,
        0.05555556],
       [0.11111111, 1.        , 0.        , ..., 0.12171612, 0.05407381,
        0.05555556],
       [0.        , 0.        , 1.        , ..., 0.        , 0.06362848,
        0.06537205],
       ...,
       [0.06085806, 0.12171612, 0.        , ..., 1.        , 0.05923489,
        0.06085806],
       [0.05407381, 0.05407381, 0.06362848, ..., 0.05923489, 1.        ,
        0.10814761],
       [0.05555556, 0.05555556, 0.06537205, ..., 0.06085806, 0.10814761,
        1.        ]])

In [147]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
       
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(retro_raw_flatten.Title)[i])
        
    return recommended_movies

In [148]:
recommendations('Angel On My Shoulder')

['My Man Godfrey',
 'Flirting with Fate',
 "Charlie Chaplin's A Burlesque On Carmen",
 'Texas Terror',
 'Million Dollar Kid',
 'Life With Father',
 'Barefoot Boy',
 'Galloping Romeo',
 'Manfish',
 'The Outlaw']